In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden MediaPipe With Image Classification

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_mediapipe_image_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_mediapipe_image_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>                                                                                               <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_mediapipe_image_classification.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

This notebook demonstrates how to use [MediaPipe](https://developers.google.com/mediapipe) in Vertex AI Model Garden.

### Objective

* Train new models
  * Convert input data to training formats
  * Create [custom jobs](https://cloud.google.com/vertex-ai/docs/training/create-custom-job) to train new models
  * Find and export best models

* Cleanup resources

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

### Colab Only
Run the following commands to install dependencies and to authenticate with Google Cloud if running on Colab.

In [ ]:
! pip3 install --upgrade pip

if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).
1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

In [ ]:
import json
import os
from datetime import datetime

import tensorflow
from google.cloud import aiplatform

now = datetime.now().strftime("%Y%m%d-%H%M%S")

# The project and bucket are for experiments below.
PROJECT_ID = ""  # @param {type:"string"}
# The form for BUCKET_URI is gs://<bucket-name>.\n",
BUCKET_URI = "gs://"  # @param {type:"string"}

# You can choose a region from https://cloud.google.com/about/locations.
# Only regions prefixed by "us", "asia", or "europe" are supported.
REGION = "us-central1"  # @param {type:"string"}
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} is not supported. It must be prefixed by "us", "asia", or "europe".'

! gcloud config set project $PROJECT_ID

STAGING_BUCKET = os.path.join(BUCKET_URI, "temp/%s" % now)

EVALUATION_RESULT_OUTPUT_DIRECTORY = os.path.join(STAGING_BUCKET, "evaluation")
EVALUATION_RESULT_OUTPUT_FILE = os.path.join(
    EVALUATION_RESULT_OUTPUT_DIRECTORY, "evaluation.json"
)

EXPORTED_MODEL_OUTPUT_DIRECTORY = os.path.join(STAGING_BUCKET, "model")
EXPORTED_MODEL_OUTPUT_FILE = os.path.join(
    EXPORTED_MODEL_OUTPUT_DIRECTORY, "model.tflite"
)

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

### Define training machine specs

In [ ]:
TRAINING_JOB_DISPLAY_NAME = "mediapipe_image_classifier_%s" % now
TRAINING_CONTAINER = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/mediapipe-train"
TRAINING_MACHINE_TYPE = "n1-highmem-16"
TRAINING_ACCELARATOR_TYPE = "NVIDIA_TESLA_V100"
TRAINING_ACCELERATOR_COUNT = 2

### Prepare input data for training

Retraining a model for image classification requires a dataset that includes all kinds of items, or classes, that you want the completed model to be able to identify. You can do this by trimming down a public dataset to only the classes that are relevant to your usecase, compiling your own data, or some combination of both. The dataset can be significantly smaller that what would be required to train a new model. For example, the [ImageNet](https://www.image-net.org/) dataset used to train many reference models contains millions of images with thousands of categories. Transfer learning with Model Maker can retrain an existing model with a smaller dataset and still perform well, depending on your inference accuracy goals.

You can re-use an existing dataset such as `gs://cloud-samples-data-us-central1/vision/automl_classification/flowers` to re-train the model or you can upload your own dataset to GCS. If you are using your own dataset, ensure that your image directory contains several subdirectories, each corresponding to specific class labels. Your training data should also follow this pattern: <image_path>/<label_name>/<image_names>.*.

In [ ]:
training_data_path = "gs://cloud-samples-data-us-central1/vision/automl_classification/flowers"  # @param {type:"string"}
split_ratio = "0.6,0.1,0.1"  # @param {type:"string"}

### Set retraining options

There are a few required settings to run a retraining aside from your training dataset: output directory for the model and the model architecture. Use HParams object export_dir parameter to specify a model output directory. Use the SupportedModels class to specify the model architecture. The image classifier solution supports the following model architectures:

*   MobileNet-V2
*   EfficientNet-Lite0
*   EfficientNet-Lite2
*   EfficientNet-Lite4

To set the required parameters, use the following code:

In [ ]:
model_architecture = "mobilenet_v2"  # @param ["mobilenet_v2", "efficientnet_lite0", "efficientnet_lite2", "efficientnet_lite4"]

# The learning rate to use for gradient descent training.
learning_rate: float = 0.01  # @param {type:"number"}
#  Batch size for training.
batch_size: int = 2  # @param {type:"number"}
# Number of training iterations over the dataset.
epochs: int = 10  # @param {type:"slider", min:0, max:100, step:1}
# If true, the base module is trained together with the classification layer on
# top.
do_fine_tuning: bool = False  # @param {type:"boolean"}
# A regularizer that applies a L1 regularization penalty.
l1_regularizer: float = 0.0  # @param {type:"number"}
# A regularizer that applies a L2 regularization penalty.
l2_regularizer: float = 0.0001  # @param {type:"number"}
# Amount of label smoothing to apply. See tf.keras.losses for more details.
label_smoothing: float = 0.1  # @param {type:"number"}
# A boolean controlling whether the training dataset is augmented by randomly
# distorting input images, including random cropping, flipping, etc. See
# utils.image_preprocessing documentation for details.
do_data_augmentation: bool = True  # @param {type:"boolean"}
# Number of training samples used to calculate the decay steps
# and create the training optimizer.
decay_samples: int = 2560000  # @param {type:"number"}
# Number of warmup steps for a linear increasing warmup schedule on learning
# rate. Used to set up warmup schedule by model_util.WarmUp.
warmup_epochs: int = 2  # @param {type:"number"}

#### Run retraining
With your training dataset and retraining options prepared, you are ready to start the retraining process. This process is resource intensive and can take a few minutes to a few hours depending on your available compute resources. Using a Google Colab environment with standard CPU processing, the example retraining below takes about 20 minutes to train on approximately 4000 images. You can typically decrease your training time by using GPU processors.

To begin the retraining process on Vertex AI, use the following code:


In [ ]:
model_export_path = EXPORTED_MODEL_OUTPUT_DIRECTORY
evaluation_result_path = EVALUATION_RESULT_OUTPUT_DIRECTORY

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAINING_MACHINE_TYPE,
            "accelerator_type": TRAINING_ACCELARATOR_TYPE,
            "accelerator_count": TRAINING_ACCELERATOR_COUNT,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAINING_CONTAINER,
            "command": [],
            "args": [
                "--task_name=image_classifier",
                "--training_data_path=%s" % training_data_path,
                "--model_export_path=%s" % model_export_path,
                "--evaluation_result_path=%s" % evaluation_result_path,
                "--split_ratio=%s" % split_ratio,
                "--model_architecture=%s" % model_architecture,
                "--hparams=%s"
                % json.dumps(
                    {
                        "learning_rate": learning_rate,
                        "batch_size": batch_size,
                        "epochs": epochs,
                        "do_fine_tuning": do_fine_tuning,
                        "l1_regularizer": l1_regularizer,
                        "l2_regularizer": l2_regularizer,
                        "label_smoothing": label_smoothing,
                        "do_data_augmentation": do_data_augmentation,
                        "decay_samples": decay_samples,
                        "warmup_epochs": warmup_epochs,
                    }
                ),
            ],
        },
    }
]

training_job = aiplatform.CustomJob(
    display_name=TRAINING_JOB_DISPLAY_NAME,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

training_job.run()

### Evaluate performance

After retraining the model, we evaluate the training result on a test dataset, which is typically a portion of your original dataset not used during training. Accuracy levels between 0.8 and 0.9 are generally considered very good, but your use case requirements may differ. You should also consider how fast the model can produce an inference. Higher accuracy frequently comes at the cost of longer inference times.


In [ ]:
def get_evaluation_result(evaluation_result_path):
    try:
        with tensorflow.io.gfile.GFile(evaluation_result_path, "r") as input_file:
            evalutation_result = json.loads(input_file.read())
        return evalutation_result["accuracy"], evalutation_result["loss"]
    except:
        print(
            "Evaluation result not found. Your training dataset is likely "
            + "empty. You can adjust the size of your training dataset or adjust "
            + "how you split your dataset."
        )
        return None


evaluation_result = get_evaluation_result(EVALUATION_RESULT_OUTPUT_FILE)

if evaluation_result is not None:
    print("Accuracy:", evaluation_result[0])
    print("Loss:", evaluation_result[1])

## Export model
After retraining and evluation a model, you can save the Tensorflow Lite model, try it out in the [Image Classification](https://mediapipe-studio.webapps.google.com/demo/image_classifier) demo in MediaPipe Studio or integrate it with your application by following the [Image classification task guide](https://developers.google.com/mediapipe/solutions/vision/image_classifier). The exported model contains the generates required model metadata, as well as a classification label file.

In [ ]:
def copy_model(model_source, model_dest):
    ! gsutil cp {model_source} {model_dest}

copy_model(EXPORTED_MODEL_OUTPUT_FILE, "image_classification_model.tflite")

if "google.colab" in str(get_ipython()):
    from google.colab import files

    files.download("image_classification_model.tflite")

## Clean up

In [ ]:
# Delete training data and jobs.
if training_job.list(filter=f'display_name="{TRAINING_JOB_DISPLAY_NAME}"'):
    training_job.delete()

!gsutil rm -r {STAGING_BUCKET}